# Projet

On utlisera l'API de Github

Requetes utiles :
- Recuperer les répertoires en top tendences sur github
- Recuperer les X répertoires en tendences dans toutes les languages
- Recuperer les X répertoires les plus "starred" pour chaque languages
- Recuperer les X répertoires les plus "starred" pour une collection
- Recuperer les collections contenant un mot
- Recuperer les répertoires d'une personne
- Recuperer les répertoires appartenant à une personne (pas fork)
- Recuperer les répertoires forkés par une personne
- Recuperer les personnes ayant forkés un répertoires
- Recuperer les personnes ayant "starred" un repertoire

- on ne gere pas les watchers, c'est un peu inutile comme fonctionnalité

In [3]:
!pip install 'pymongo[srv]'

In [9]:
from pymongo import MongoClient

client = MongoClient("mongo")
database = client['projet']

# database.command("serverStatus")

###  Utilisateur

* Infos d'un user
* Repos d'un user
* Gist d'un user
* Stars du user
* Repos les plus populaires (les + vus) : voir si possible
* Repos les plus forks
* Repos le plus starred
* Forks d'un user
* Répos owns du user

#### Informations d'un utilisateur

L'endpoint pour un utilisateur est https://api.github.com/users/{utilisateur}.

In [66]:
# Création des collections permettant la manipulation optimale de l'API Github.

collection_users = database.users
collection_topics = database.topics
collection_repos = database.repositories # On sait jamais

In [67]:
endpoints = {
    'user': {
        '_': lambda user: 'https://api.github.com/users/' + user,
        'following': lambda user: 'https://api.github.com/users/' + user + '/following'
    },
    'users': 'https://api.github.com/users'
}

In [68]:
import types
from requests import get

# exemple : 
# getEndpoint('user.following', 'mexanga')
def getEndpoint(endpoint, *arguments):
    splitOfEndpoints = endpoint.split('.')
    cursorEndpoint = None
    
    for cursorSplit in splitOfEndpoints:
        try:
            if (None == cursorEndpoint): cursorEndpoint = endpoints[cursorSplit]
            else: cursorEndpoint = cursorEndpoint[cursorSplit]
        except:
            return None
        
    if type(cursorEndpoint) is types.LambdaType: cursorEndpoint = cursorEndpoint(*arguments)
    
    return cursorEndpoint

# exemple : 
# requestEndpoint('user.following', 'mexanga')
def requestEndpoint(endpoint, *argument):
    endpoint = getEndpoint(endpoint, *argument)
    content = get(endpoint).json()
    return content

[{'login': 'cgrandval',
  'id': 903317,
  'node_id': 'MDQ6VXNlcjkwMzMxNw==',
  'avatar_url': 'https://avatars.githubusercontent.com/u/903317?v=4',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/cgrandval',
  'html_url': 'https://github.com/cgrandval',
  'followers_url': 'https://api.github.com/users/cgrandval/followers',
  'following_url': 'https://api.github.com/users/cgrandval/following{/other_user}',
  'gists_url': 'https://api.github.com/users/cgrandval/gists{/gist_id}',
  'starred_url': 'https://api.github.com/users/cgrandval/starred{/owner}{/repo}',
  'subscriptions_url': 'https://api.github.com/users/cgrandval/subscriptions',
  'organizations_url': 'https://api.github.com/users/cgrandval/orgs',
  'repos_url': 'https://api.github.com/users/cgrandval/repos',
  'events_url': 'https://api.github.com/users/cgrandval/events{/privacy}',
  'received_events_url': 'https://api.github.com/users/cgrandval/received_events',
  'type': 'User',
  'site_admin': False},
 {'login': 'd

### Topics

* Topics existant
* Topics contenant un mot
* Tags associés à un topic => Pour chaque repo, on recup les tags et on compte le nombre qu'un tag apparaît (soit aggregate du début à la fin normalement)
* Repos d'un topic
* Languages le plus utilisés dans un topic : voir si c'est possible